# A Trial Attempt for the Course Project

We have 2 tasks, as described below:

In [207]:
# PROBLEM MODEL

using JuMP, Clp

m = Model(solver=ClpSolver())

# Number of days in a week
num_days = 7

# Divide each day 10:30am to 11:30pm into 15 minute slots. So there are 13*4 = 52 slots in each day.
num_slots = 13*4

# @variable(m, slots[1:num_days, 1:num_slots] >= 0)

52

In [208]:
# TASK 1: Research

# Whether we are doing research in a given slot, binary value
@variable(m, 0 <= research[1:num_days, 1:num_slots] <= 1)

# # Populate the variable
# for i=1:num_days
#     for j=1:num_slots
#         @constraint(m, research[i, j] == (slots[i, j] == 1))
#     end
# end

# Deadline is Friday, so incorporate that
# @expression(m, time_spent_on_research, sum( sum(research[i, j] for j=1:num_slots) for i=1:num_days))
@expression(m, time_spent_research, sum( sum(research[i, j] for j=1:num_slots) for i=1:4))
min_hours_research =12
@constraint(m, time_spent_research >= min_hours_research*4)

@variable(m, num_hours_worked_research[1:num_days])
for i=1:num_days
    @constraint(m, num_hours_worked_research[i] == sum(research[i, j] for j=1:num_slots)/4)
end


In [209]:
# TASK 2: Edge Computing Course

# Whether we are doing edge_computing a given slot, binary value
@variable(m, 0 <= edge_computing[1:num_days, 1:num_slots] <= 1)

# # Populate the variable
# for i=1:num_days
#     for j=1:num_slots
#         @constraint(m, edge_computing[i, j] == (slots[i, j] == 2))
#     end
# end

@expression(m, time_spent_edge_computing, sum( sum(edge_computing[i, j] for j=1:num_slots) for i=1:num_days))

# Deadline is Friday 4pm, so incorporate that
deadline_day = 5
deadline_slot = (32-21)*2 + 1
for j=deadline_slot:num_slots
    @constraint(m, edge_computing[deadline_day, j] == 0)
end

# Make some progress by the end of Wednesday, spend at least 3 hours
@constraint(m, sum(edge_computing[3, j] for j=1:num_slots) >= 3*4)

# not worth doing work on Saturday and Sunday
for i=6:7
    for j=1:num_slots
        @constraint(m, edge_computing[i, j] == 0)
    end
end
min_hours_edge_computing = 8
@constraint(m, time_spent_edge_computing >= min_hours_edge_computing*4)

@variable(m, num_hours_worked_edge_computing[1:num_days])
for i=1:num_days
    @constraint(m, num_hours_worked_edge_computing[i] == sum(edge_computing[i, j] for j=1:num_slots)/4)
end

In [210]:
# TEMPORARY: Monday and Tuesday have already passed
for i=1:2
    for j=1:num_slots
        @constraint(m, research[i, j] == 0)
        @constraint(m, edge_computing[i, j] == 0)
    end
end

In [211]:
# Constraint: Should take break for 15 minutes after every hour of work. Every 5th slot is a break.
for i=1:num_days
    for j=1:Int(floor(num_slots/5))
        @constraint(m, research[i, j*5] == 0)
        @constraint(m, edge_computing[i, j*5] == 0)
    end
end

In [212]:
# Constraint: Can only do one type of task in each slot
for i=1:num_days
    for j=1:num_slots
        @constraint(m, research[i,j] + edge_computing[i, j] <= 1)
    end
end

In [213]:
# Objective: Minimize context switches from one task to another, and do not have long working days

# Context switches for research
@variable(m, 0 <= context_switches_research[1:num_days, 1:(num_slots-1)] <= 1)

for i=1:num_days
    for j=2:num_slots
        @constraint(m, context_switches_research[i, j-1] >= (research[i, j] - research[i, j-1]))
        @constraint(m, context_switches_research[i, j-1] >= (research[i, j-1] - research[i, j]))
    end
end

@expression(m, num_context_switches_research, sum( sum(context_switches_research[i, j] for j=1:(num_slots-1)) for i=1:num_days))

# Context switches for edge computing
@variable(m, 0 <= context_switches_edge[1:num_days, 1:(num_slots-1)] <= 1)

for i=1:num_days
    for j=2:num_slots
        @constraint(m, context_switches_edge[i, j-1] >= (edge_computing[i, j] - edge_computing[i, j-1]))
        @constraint(m, context_switches_edge[i, j-1] >= (edge_computing[i, j-1] - edge_computing[i, j]))
    end
end

@expression(m, num_context_switches_edge, sum( sum(context_switches_edge[i, j] for j=1:(num_slots-1)) for i=1:num_days))

# Do not have long working days unnecessarily
preferred_hours_per_day = 8
@variable(m, hours_exceeded[1:num_days] >= 0)
for i=1:num_days
    @constraint(m, hours_exceeded[i] >= (num_hours_worked_research[i]+num_hours_worked_edge_computing[i] - preferred_hours_per_day))
end
@expression(m, total_hours_exceeded, sum(hours_exceeded[i] for i=1:num_days))
@objective(m, Min, num_context_switches_research + num_context_switches_edge + total_hours_exceeded)
status = solve(m)
println(status)

Optimal


In [214]:
research = getvalue(research)
# println(size(research))
start_time = 1030
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

println("Research Schedule")
for i=1:7
    println(days[i], ": ", sum(research[i, j] for j=1:num_slots)/4, " hours")
#     for j=1:num_slots
#         println(start_time+(j-1)*15, ": ", research[i, j])
#     end
end
println()
println("Total Research Hours: ", sum( sum(research[i, j] for j=1:num_slots) for i=1:7)/4)

Research Schedule
Monday: 0.0 hours
Tuesday: 0.0 hours
Wednesday: 5.000000000000002 hours
Thursday: 7.0000000000000036 hours
Friday: 0.0 hours
Saturday: 0.0 hours
Sunday: 0.0 hours

Total Research Hours: 12.000000000000005


In [215]:
edge_computing = getvalue(edge_computing)
# println(size(research))
start_time = 1030
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

println("Edge Computing Schedule")
for i=1:7
    println(days[i], ": ", sum(edge_computing[i, j] for j=1:num_slots)/4, " hours")
#     for j=1:num_slots
#         println(start_time+(j-1)*15, ": ", edge_computing[i, j])
#     end
end
println()
println("Total Edge Computing Hours: ", sum( sum(edge_computing[i, j] for j=1:num_slots) for i=1:7)/4)

Edge Computing Schedule
Monday: 0.0 hours
Tuesday: 0.0 hours
Wednesday: 3.0 hours
Thursday: 1.0 hours
Friday: 3.9999999999999973 hours
Saturday: 0.0 hours
Sunday: 0.0 hours

Total Edge Computing Hours: 7.999999999999997
